
# Prácticas: Predicción puntual e intervalos de confianza con `statsmodels`

**Objetivo.** En este notebook trabajarás con regresión lineal clásica usando el conjunto de datos real **Longley** (incluido en `statsmodels`) para:
- Estimar un modelo OLS.
- Obtener **predicción puntual** del **valor esperado** \(\mathbb{E}[Y\mid X]\) y del **valor individual** (nueva observación).
- Construir **intervalos de confianza** para el valor esperado y **intervalos de predicción** para una nueva observación.
- Visualizar y comparar ambas bandas de incertidumbre.
- Realizar predicciones para **nuevos datos**.

> Nota: Los intervalos para el valor esperado son más estrechos que los de predicción, ya que estos últimos incluyen la variabilidad del error aleatorio.


## 1. Preparación del entorno

In [ ]:

# --- Importación de librerías ---
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from IPython.display import display

# Ajustes generales
pd.set_option('display.float_format', '{:,.3f}'.format)


## 2. Cargar datos reales: Longley

In [ ]:

# Conjunto clásico de datos económicos de EE.UU. (1947–1962)
data = sm.datasets.longley.load_pandas().data

print("Dimensiones:", data.shape)
display(data.head())

# Variable dependiente y explicativas
y = data['TOTEMP']          # Empleo total (objetivo)
X = data.drop(columns=['TOTEMP'])  # Explicativas
X = sm.add_constant(X)      # Añadimos término independiente


## 3. Ajuste del modelo OLS

In [ ]:

modelo = sm.OLS(y, X).fit()
print(modelo.summary())


## 4. Predicción e intervalos (observaciones existentes)

In [ ]:

# Seleccionamos, por ejemplo, las 3 primeras filas para ilustrar
X_pred = X.iloc[:3]
predicciones = modelo.get_prediction(X_pred)
pred_summary = predicciones.summary_frame(alpha=0.05)  # 95%

print("Predicciones y bandas (95%):")
display(pred_summary)

# Columnas clave:
# - mean: predicción puntual del valor esperado E[Y|X]
# - mean_ci_lower, mean_ci_upper: IC para el valor esperado
# - obs_ci_lower, obs_ci_upper: intervalo de predicción para una nueva observación


## 5. Visualización de la incertidumbre (un caso)

In [ ]:

# Elegimos el primer caso para graficar
i = pred_summary.index[0]
mean_pred = pred_summary.loc[i, 'mean']
mean_ci_low = pred_summary.loc[i, 'mean_ci_lower']
mean_ci_up  = pred_summary.loc[i, 'mean_ci_upper']
obs_ci_low  = pred_summary.loc[i, 'obs_ci_lower']
obs_ci_up   = pred_summary.loc[i, 'obs_ci_upper']

plt.figure(figsize=(6,4))

# IC del valor esperado
plt.errorbar(x=['Valor esperado'], y=[mean_pred],
             yerr=[[mean_pred - mean_ci_low], [mean_ci_up - mean_pred]],
             fmt='o', capsize=5, label='IC valor esperado (95%)')

# Intervalo de predicción (valor individual futuro)
plt.errorbar(x=['Valor observado'], y=[mean_pred],
             yerr=[[mean_pred - obs_ci_low], [obs_ci_up - mean_pred]],
             fmt='o', capsize=5, label='Intervalo de predicción (95%)')

plt.title("Comparación de intervalos (caso 1)")
plt.ylabel("Predicción de empleo (TOTEMP)")
plt.legend()
plt.show()


## 6. Predicción para nuevos datos

In [ ]:

# Construimos un punto fuera de la muestra (ajusta los valores a conveniencia)
nuevo_X = pd.DataFrame({
    'const': [1.0],
    'GNPDEFL': [90.0],
    'GNP': [470.0],
    'UNEMP': [5.0],
    'ARMED': [230.0],
    'POP': [120.0],
    'YEAR': [1963.0]
})

pred_nuevo = modelo.get_prediction(nuevo_X).summary_frame(alpha=0.05)
print("Predicción para un nuevo punto (95%):")
display(pred_nuevo)



## 7. Ejercicios propuestos

1. **Cambiar el punto nuevo**: construye varios escenarios nuevos (p. ej., distintos valores de `UNEMP` y `GNP`) y compara cómo se ensanchan o estrechan las bandas.
2. **Diagnóstico**: utiliza `modelo.get_influence()` para calcular residuos studentizados y leverage. Identifica posibles observaciones influyentes.
3. **Validación cruzada simple**: separa 3-4 observaciones como "test" y evalúa el error de predicción fuera de muestra.
4. **Multicolinealidad**: calcula el VIF para las variables explicativas y comenta posibles efectos en la inferencia.
5. **Otra base de datos**: repite el flujo con otro dataset de `statsmodels` (por ejemplo, `sm.datasets.grunfeld`) y compara resultados.


## 8. (Opcional) Herramientas de diagnóstico

In [ ]:

# --- Cálculo del VIF (Variance Inflation Factor) ---
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Para VIF, usamos solo X sin la constante en el cálculo y luego la volvemos a añadir para el modelo si es necesario.
X_vif = data.drop(columns=['TOTEMP']).copy()
X_vif_const = sm.add_constant(X_vif)

vif_df = pd.DataFrame({
    'variable': X_vif_const.columns,
    'VIF': [variance_inflation_factor(X_vif_const.values, i) for i in range(X_vif_const.shape[1])]
})
display(vif_df)

# --- Medidas de influencia ---
influence = modelo.get_influence()
summ = influence.summary_frame()
display(summ[['standard_resid', 'student_resid', 'hat_diag', 'cooks_d']].head())



---

### Conclusiones
- **Predicción puntual**: `prediction.summary_frame()['mean']` entrega \(\widehat{\mathbb{E}}[Y\mid X]\).
- **IC para el valor esperado**: columnas `mean_ci_lower` y `mean_ci_upper`.
- **Intervalo de predicción (valor individual)**: columnas `obs_ci_lower` y `obs_ci_upper`.
- Los intervalos de predicción son **más amplios** al incluir la variabilidad del error.

> Función clave: `results.get_prediction(Xnew).summary_frame(alpha)`.



## 9. Visualización comparativa global de bandas (valor esperado vs. predicción)

En esta sección representamos, para **todas** las observaciones, la predicción puntual, el **IC del valor esperado** y el **intervalo de predicción**.  
Ordenamos por `YEAR` para facilitar la lectura temporal (puedes cambiar la variable del eje X si lo prefieres).


In [ ]:

# Predicciones para todas las observaciones
pred_all = modelo.get_prediction(X).summary_frame(alpha=0.05).copy()

# Construimos un DataFrame auxiliar con YEAR como eje horizontal
viz = pred_all.copy()
viz['YEAR'] = X['YEAR'].values
viz['y_obs'] = y.values
viz = viz.sort_values('YEAR').reset_index(drop=True)

# Gráfico 1: bandas a lo largo del tiempo (YEAR)
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))

# Banda de intervalo de predicción (más ancha)
plt.fill_between(viz['YEAR'], viz['obs_ci_lower'], viz['obs_ci_upper'], alpha=0.2, label='Intervalo de predicción (95%)')

# Banda de IC del valor esperado (más estrecha)
plt.fill_between(viz['YEAR'], viz['mean_ci_lower'], viz['mean_ci_upper'], alpha=0.4, label='IC del valor esperado (95%)')

# Línea de la predicción media
plt.plot(viz['YEAR'], viz['mean'], label='Predicción media')

# Observaciones reales
plt.scatter(viz['YEAR'], viz['y_obs'], s=20, label='Observado')

plt.xlabel('YEAR')
plt.ylabel('TOTEMP')
plt.title('Comparación de bandas: valor esperado vs. predicción')
plt.legend()
plt.tight_layout()
plt.show()



**Lectura del gráfico**
- La **banda más clara** (intervalo de predicción) es más amplia: incluye la variabilidad del error individual.  
- La **banda más oscura** (IC de la media) es más estrecha: solo recoge la incertidumbre de la estimación del valor esperado.  
- Los puntos muestran los valores **observados**. Cuando los puntos caen fuera de la banda de predicción, puede indicar observaciones atípicas o que el modelo es demasiado restrictivo.
